<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import nltk
from datetime import timedelta
import os
from nltk.corpus import stopwords
import numpy as np
import sqlite3
from collections import Counter, defaultdict
from string import punctuation
import re
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
sw = stopwords.words('english')
import janitor
import spacy
from textblob import TextBlob
tb = TextBlob('')
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load('en_core_web_lg')
import seaborn as sns
import scipy.stats as stats
from scipy.stats import kruskal
from gensim.models import CoherenceModel
from gensim import corpora
from gensim.models.ldamodel import LdaModel
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import interact_manual, interactive_output
from IPython.display import display, clear_output
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import spacy
import math
from textblob import TextBlob
tb = TextBlob('')
from spacytextblob.spacytextblob import SpacyTextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelEncoder
from random import randint
from sklearn.model_selection import RandomizedSearchCV
import winsound
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from spellchecker import SpellChecker
from textblob import TextBlob
import eli5
from eli5.sklearn import PermutationImportance
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    nltk.download('punkt')
    nltk.download('words')
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import words
from nltk.util import ngrams
import random
from wordcloud import WordCloud
import xgboost as xgb
import matplotlib.dates as mdates

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aidan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\aidan\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
def get_most_common_words(texts, n_top=10):
    vectorizer = CountVectorizer(stop_words='english')
    word_matrix = vectorizer.fit_transform(texts)
    sum_words = word_matrix.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:n_top]


In [3]:
def analyze_agency_involvement(data, agency_list):
    results = {}
    
    for agency in agency_list:
        agency_data = {
            'involvement_percentage': (data[agency].mean()) * 100,
            'text_analysis': {},
            'model_performance': {},
            'feature_importance': {}
        }
        
        # Splitting text based on agency involvement
        agency_present_text = data[data[agency] == 1]['combined_text'].dropna()
        agency_absent_text = data[data[agency] == 0]['combined_text'].dropna()
        
        # Common words analysis
        common_words_present = get_most_common_words(agency_present_text)
        common_words_absent = get_most_common_words(agency_absent_text)
        
        # Relative frequency analysis
        total_words_present = sum(freq for _, freq in common_words_present)
        total_words_absent = sum(freq for _, freq in common_words_absent)
        relative_freq_present = {word: freq / total_words_present for word, freq in common_words_present}
        relative_freq_absent = {word: freq / total_words_absent for word, freq in common_words_absent}
        
        # Comparative frequency
        comparative_freq = {word: relative_freq_present.get(word, 0) - relative_freq_absent.get(word, 0) for word in set(relative_freq_present) | set(relative_freq_absent)}
        sorted_comparative_freq = sorted(comparative_freq.items(), key=lambda x: abs(x[1]), reverse=True)
        
        agency_data['text_analysis'] = {
            'common_words_present': common_words_present,
            'common_words_absent': common_words_absent,
            'relative_freq_present' : relative_freq_present,
            'relative_freq_absent' : relative_freq_absent,
            'sorted_comparative_freq': sorted_comparative_freq
        }
        
        # Model training and evaluation
        X = tfidf_vectorizer.fit_transform(data['combined_text'].fillna(''))
        y = data[agency]
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        lr_model = LogisticRegression(max_iter=1000)
        lr_model.fit(X_train, y_train)
        
        y_pred = lr_model.predict(X_test)
        
        # Storing model performance
        agency_data['model_performance'] = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred)
        }
        
        # Feature importance
        feature_names = tfidf_vectorizer.get_feature_names_out()
        coefficients = lr_model.coef_[0]
        features_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})
        features_sorted = features_df.sort_values(by='Coefficient', ascending=False)
        
        agency_data['feature_importance'] = {
            'positive': features_sorted.head(10).to_dict('records'),
            'negative': features_sorted.tail(10).to_dict('records')
        }
        
        # Store results for current agency
        results[agency] = agency_data
    
    return results

In [4]:
def get_agency_stats(df = flat, agency = 'nps'):
    f_rows = df[df['agency'] == agency]
    
    agency_rf_pres = f_rows['relative_freq_present'].iloc[0]
    agency_rf_abs = f_rows['relative_freq_absent'].iloc[0]
    agency_involvement_percentage = f_rows['involvement_percentage']
    agency_common_words_present = f_rows['common_words_present']
    agency_common_words_absent = f_rows['common_words_absent']
    agency_relative_freq_present = f_rows['relative_freq_present']
    agency_relative_freq_absent = f_rows['relative_freq_absent']
    agency_sorted_comparative_freq = f_rows['sorted_comparative_freq']
    agency_accuracy = f_rows['accuracy']
    agency_precision = f_rows['precision']
    agency_recall = f_rows['recall']
    agency_f1 = f_rows['f1']
    agency_positive = f_rows['positive']
    agency_negative = f_rows['negative']

    # Return all extracted values
    return {
        'involvement_percentage': agency_involvement_percentage,
        'common_words_present': agency_common_words_present,
        'common_words_absent': agency_common_words_absent,
        'relative_freq_present': agency_relative_freq_present,
        'relative_freq_absent': agency_relative_freq_absent,
        'sorted_comparative_freq': agency_sorted_comparative_freq,
        'accuracy': agency_accuracy,
        'precision': agency_precision,
        'recall': agency_recall,
        'f1': agency_f1,
        'positive': agency_positive,
        'negative': agency_negative
    }
    return agency_rf_pres, agency_rf_abs

def pretty_print_results(results):
    # Initialize PrettyPrinter
    pp = pprint.PrettyPrinter(indent=4)
    
    print("Agency Stats Overview:\n")
    
    # Table header
    print(f"{'Metric':<25} | {'Value'}")
    print("-" * 50)
    
    # Loop through each result, print formatted string for numbers and use pprint for complex types
    for key, value in results.items():
        if isinstance(value.iloc[0], (float, int)):
            # Format numbers to two decimal places if it's a float/int
            print(f"{key:<25} | {value.iloc[0]:.2f}")
        else:
            # Use pretty print for dictionaries, lists, etc.
            print(f"{key:<25} :")
            pp.pprint(value.iloc[0])
            print("-" * 50)

NameError: name 'flat' is not defined